# Dynamics of a Heated Tank with PI Temperature Control
This is the tenth problem of the famous set of [Ten Problems in Chemical Engineering](https://www.polymath-software.com/ASEE/Tenprobs.pdf).  Here, the goal is to solve ordinary differential equations, generation of step functions, simulation of a proportionalintegral controller.


# Problem Setup


# Problem Tasks

(a)    Demonstrate the open loop performance (set Kc = 0) of this system when the system is ini-tially operating at design steady state at a temperature of 80°C, and inlet temperature Ti is sud-denly changed to 40°C at time t = 10 min. Plot the temperatures T, T0, and Tm to steady state,and  verify  that  Padé  approximation  for  1  min  of  dead  time  given  in  Equation  (42)  is  workingproperly.

b)    Demonstrate the closed loop performance of the system for the conditions of part (a) and thebaseline parameters from Table (4). Plot temperatures T, T0, and Tm to steady state.

c)Repeat part (b) with Kc = 500 kJ/min⋅°C.

d)Repeat part (c) for proportional only control action by setting the term Kc/τI = 0.

e)Implement limits on q (as per Equation (47)) so that the maximum is 2.6 times the baselinesteady  state  value  and  the  minimum  is  zero.  Demonstrate  the  system  response  from  baselinesteady state for a proportional only controller when the set point is changed from 80°C to 90°C att = 10 min. Kc = 5000 kJ/min⋅°C. Plot q and qlim versus time to steady state to demonstrate the lim-its. Also plot the temperatures T, T0, and Tm to steady state to indicate controller performance 

# Solutions